In [ ]:
#r "D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

## Open Database

In [ ]:
BoSSSshell.WorkflowMgm.Init("CounterFlowFlame_MF_FullComparison_bugfixed155");
static var myBatch = GetDefaultQueue();
(myBatch as MsHPC2012Client).ComputeNodes = new string[]{"HPCLUSTER3"};

static var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

In [ ]:
// BoSSSshell.WorkflowMgm.Init("CounterFlowFlame_MF_Comparison2morexcells");
// static var myBatch = GetDefaultQueue();
// BoSSSshell.WorkflowMgm.DefaultDatabase = myDb;

In [ ]:
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation()

## Case configuration

In [ ]:
int nCells = 10;
// int[] multiplierS = new int[]{2,5,11}; // multiplying factor of inlet velocities
int[] multiplierS = new int[]{5}; // multiplying factor of inlet velocities
// double[] Smoothings = new double[] {0};
double[] Smoothings = new double[] {0,200,100,50};
int dgMF = 3; // Mass fraction DG degree
int[] DGDegrees = new int[] {4};

int[] nCellsArray = new int[] {10};

double[] xCellsMultiplier = new double[] {1};

double InitialMassFuelIn = 0.02400; //kg/m2s 
double InitialMassAirIn = 0.02400 *2; //kg/m2s //////////////////////////////////////*3
bool parabolicVelocityProfile = false;
bool chemicalReactionActive = true; 
bool useFullGeometry = false;
bool wallBounded = true;

## Create grid

In [ ]:
public static class GridFactory {

    public static Grid2D GenerateGrid(int nCells, bool wallBounded, double xDensity) {
        double L = 0.02;
        double LRef = L;
        double R_dim = L / 2;
        double separation = 1.0; // nondimensional
        double xleft = 0;
        double xright = separation;
        double radius_inlet = R_dim / LRef;
        double ybot = -separation * 3; 
        double ytop = separation * 3; 

        // Define Grid
        double[] _xNodes;
        double[] _yNodes;

       // X-NODES
        double xNodesDensity = xDensity; 
        _xNodes = GenericBlas.Linspace(xleft, xright, (int)(nCells * xNodesDensity + 1));
        double sf3 = 0.80;
        var myCenterNodes = GenericBlas.SinLinSpacing(0, radius_inlet*2,0.7, nCells*2 + 1).ToList(); // center
        var centerNodes = myCenterNodes.GetSubVector(0, myCenterNodes.Count / 2 + 1); // Take only "bottom side" of node array


        List<double> yNodesTop = (GenericBlas.SinLinSpacing(radius_inlet, (ytop - radius_inlet) * 2 + radius_inlet, sf3, nCells * 2 + 1).ToList()); // Nodes corresponding to the oxidizer inlet, right part
        var myYnodesTop = yNodesTop.GetSubVector(0, yNodesTop.Count / 2 + 1); // Take only "bottom side" of node array

        List<double> yUpperPart = new List<double>();
        yUpperPart.AddRange(centerNodes.SkipLast(1));
        yUpperPart.AddRange(myYnodesTop.SkipLast(0));

        var yBottomPart = (yUpperPart.ToArray()).CloneAs();
        yBottomPart.ScaleV(-1.0);
        Array.Reverse(yBottomPart);
        var yNodes = new List<double>();
        yNodes.AddRange(yBottomPart.SkipLast(1));
        yNodes.AddRange(yUpperPart.SkipLast(0));
        
        var grd = Grid2D.Cartesian2DGrid(_xNodes, yNodes.ToArray());
        // Define Edge tags
        grd.EdgeTagNames.Add(1, "Velocity_Inlet_CH4");
        grd.EdgeTagNames.Add(2, "Velocity_Inlet_O2");
        grd.EdgeTagNames.Add(3, "Pressure_Outlet");

        if(wallBounded){
            grd.EdgeTagNames.Add(4, "Wall");
        }
        grd.DefineEdgeTags(delegate (double[] X) {
            double x = X[0];
            double y = X[1];
            //Edge tags
            //1: Velocity inlet O_2
            //2: Velocity inlet CH_4
            //3: Pressure outlet
            if(Math.Abs(x - xleft) < 1e-8) { // Left boundary
                if(Math.Abs(y) - radius_inlet < 1e-8) { // Fuel Inlet
                    return 1;
                } else {
                    if(wallBounded){
                    return 4;
                } else{
                    return 3;
                }
                }
            }
            if(Math.Abs(x - xright) < 1e-8) { // right boundary
                if(Math.Abs(y) - radius_inlet < 1e-8) { // oxy Inlet
                    return 2;//2
                } else {
                    if(wallBounded){
                        return 4;
                    } else{
                        return 3;
                    }
                }
            } else {
                return 3; //3 Pressure outlet
            }
        });
        myDb.SaveGrid(ref grd);

        return grd;
    }

    public static Grid2D GenerateGrid_FullGeometry(int nCells) {
        double L = 0.02;
        double LRef = L;
        double R_dim = L / 2;
        double separation = 1.0; // nondimensional
        double xleft = 0;
        double xright = separation;
        double radius_inlet = R_dim / LRef;
        double ybot = -separation * 3; //
        double ytop = separation * 3;//

        // Define Grid
        double[] _xNodes;
        double[] _yNodes;

         // X-NODES
        double[] XNODES1 = GenericBlas.Linspace(0, separation, (int)(nCells / 2 + 1));
        double[] XNODES2 = GenericBlas.Linspace(separation * 1, separation * 2, (int)(nCells + 1));
        double[] XNODES3 = GenericBlas.Linspace(separation * 2, separation * 3, (int)(nCells / 2 + 1));
        List<double> xNodesList = new List<double>();
        xNodesList.AddRange(XNODES1.Take(XNODES1.Length - 1));
        xNodesList.AddRange(XNODES2.Take(XNODES2.Length - 1));
        xNodesList.AddRange(XNODES3);
        _xNodes = xNodesList.ToArray();

        // Y-NODES
        double sf3 = 0.0;
        var _yNodes2 = GenericBlas.Linspace(-radius_inlet, radius_inlet, nCells + 1).ToList(); // center
        List<double> yNodesTop = (GenericBlas.SinLinSpacing(radius_inlet, (ytop - radius_inlet) * 2 + radius_inlet, sf3, nCells * 2 + 1).ToList()); // Nodes corresponding to the oxidizer inlet, right part
        var myYnodesTop = yNodesTop.GetSubVector(0, yNodesTop.Count / 2 + 1); // Take only "bottom side" of node array
        var myYNodesBot = myYnodesTop.CloneAs();
        myYNodesBot.ScaleV(-1.0);
        Array.Reverse(myYNodesBot);
        List<double> list2 = new List<double>();
        list2.AddRange(myYNodesBot.Take(myYNodesBot.Length - 1).ToList());
        list2.AddRange(_yNodes2.Take(_yNodes2.Count() - 1).ToList());
        list2.AddRange(myYnodesTop.ToList());
        _yNodes = list2.ToArray();

        double[] CutOut1Point1 = new double[2] { separation, separation / 2 };
        double[] CutOut1Point2 = new double[2] { 0, -separation / 2 };

        double[] CutOut2Point1 = new double[2] { 2 * separation, -separation / 2 };
        double[] CutOut2Point2 = new double[2] { 3 * separation, separation / 2 };

        var CutOut1 = new BoSSS.Platform.Utils.Geom.BoundingBox(2);
        CutOut1.AddPoint(CutOut1Point1);
        CutOut1.AddPoint(CutOut1Point2);

        var CutOut2 = new BoSSS.Platform.Utils.Geom.BoundingBox(2);
        CutOut2.AddPoint(CutOut2Point1);
        CutOut2.AddPoint(CutOut2Point2);
        var CutOuts = new BoSSS.Platform.Utils.Geom.BoundingBox[] { CutOut1, CutOut2 };
        var grd = Grid2D.Cartesian2DGrid(_xNodes, _yNodes, CutOuts: CutOuts);

        grd.EdgeTagNames.Add(1, "Velocity_Inlet_CH4");
        grd.EdgeTagNames.Add(2, "Velocity_Inlet_O2");
        grd.EdgeTagNames.Add(3, "Pressure_Outlet");
        grd.EdgeTagNames.Add(4, "Wall");

        grd.DefineEdgeTags(delegate (double[] X) {
            double x = X[0];
            double y = X[1];
            //Edge tags
            //1: Velocity inlet O_2
            //2: Velocity inlet CH_4
            //3: Pressure outlet
            if(Math.Abs(x - separation) < 1e-8 && Math.Abs(y) - radius_inlet < 1e-8) {
                return 1;
            }
            if(Math.Abs(x - 2 * separation) < 1e-8 && Math.Abs(y) - radius_inlet < 1e-8) {
                return 2;
            }
            if(Math.Abs(y - radius_inlet) < 1e-8 && x > 0 && x < separation) {
                return 4;
            }
            if(Math.Abs(y - radius_inlet) < 1e-8 && x > 2 * separation && x < 3 * separation) {
                return 4;
            }
            if(Math.Abs(y + radius_inlet) < 1e-8 && x > 0 && x < separation) {
                return 4;
            }
            if(Math.Abs(y + radius_inlet) < 1e-8 && x > 2 * separation && x < 3 * separation) {
                return 4;
            }
            return 3;
        }
        );
        myDb.SaveGrid(ref grd);

        return grd;
    }
}

In [ ]:
public static class BoundaryValueFactory { 

    public static string GetPrefixCode(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma) {
        using(var stw = new System.IO.StringWriter()) {

            stw.WriteLine("static class BoundaryValues {");

            stw.WriteLine("  static public double ConstantValue(double[] X) {");
            stw.WriteLine("    return "+ ConstVal +";");
            stw.WriteLine("  }");

            stw.WriteLine("  static public double ParabolaVelocityFuel(double[] X) {");
            stw.WriteLine("    return  (1.0 - Math.Pow(X[1] / "+inletRadius+", 2)) * "+uInFuel+" ;");
            stw.WriteLine("  }");

            stw.WriteLine("  static public double ParabolaVelocityAir(double[] X) {");
            stw.WriteLine("    return  -(1.0 - Math.Pow(X[1] / "+inletRadius+", 2)) * "+uInAir+";");
            stw.WriteLine("  }");

            stw.WriteLine("  static public double RegularizedPlugFlowFuel(double[] X) {");    
            stw.WriteLine(" double res = 0;");  
            stw.WriteLine("         if(X[1] > 0) { ");
            stw.WriteLine("     double H = 0.5 * (1.0 + Math.Tanh("+sigma+" * (X[1] - "+inletRadius+"))); ");
            stw.WriteLine("           res = "+uInFuel+" * (1 - 2*H); ");
            stw.WriteLine("      }   ");
            stw.WriteLine("  else { ");
            stw.WriteLine("      double H = 0.5 * (1.0 + Math.Tanh("+sigma+" * (X[1] + ("+inletRadius+")))); ");
            stw.WriteLine("           res = "+uInFuel+" * ( 2*H-1); ");
            stw.WriteLine("  } ");
            stw.WriteLine("return res;"); 
            stw.WriteLine("  }");


            stw.WriteLine("  static public double RegularizedPlugFlowAir(double[] X) {");         
            stw.WriteLine(" double res = 0;");
            stw.WriteLine("         if(X[1] > 0) { ");
            stw.WriteLine("     double H = 0.5 * (1.0 + Math.Tanh("+sigma+" * (X[1] - "+inletRadius+"))); ");
            stw.WriteLine("           res = "+uInAir+" * (1 - 2*H)*(-1); ");
            stw.WriteLine("      }   ");
            stw.WriteLine("  else { ");
            stw.WriteLine("      double H = 0.5 * (1.0 + Math.Tanh("+sigma+" * (X[1] + ("+inletRadius+")))); ");
            stw.WriteLine("           res = "+uInAir+" * ( 2*H-1)*(-1); ");
            stw.WriteLine("  } ");
            stw.WriteLine("return res;"); 
            stw.WriteLine("  }");


            stw.WriteLine("}"); 
            return stw.ToString();
        }
    }

    static public Formula Get_ConstantValue(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ConstantValue", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }

    static public Formula Get_ParabolaVelocityFuel(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ParabolaVelocityFuel", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }
    static public Formula Get_ParabolaVelocityAir(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ParabolaVelocityAir", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }

    static public Formula Get_RegularizedPlugFlowFuel(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.RegularizedPlugFlowFuel", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }

    static public Formula Get_RegularizedPlugFlowAir(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.RegularizedPlugFlowAir", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }

}

## Create base control file
In this ControlFile basic configuration of the CounterDiffusionFlame is defined.

In [ ]:
static XNSEC_Control GiveMeTheCtrlFile(int dg, int nCells, bool isMF, double massFuelIn, double massAirIn, bool parabolicVelocityProfile, bool UsefullGeometry, bool wallBounded, double xNodesMultiplier, double smoothing) {
    var CC = new ChemicalConstants();

    var C = isMF ? new XNSEC_MF_Control() : new XNSEC_Control();

    C.NumberOfChemicalSpecies = 4; 
    C.SetDGdegree(dg); //
    if(UsefullGeometry) {
        C.SetGrid(GridFactory.GenerateGrid_FullGeometry(nCells));  //
    } else {
        C.SetGrid(GridFactory.GenerateGrid(nCells, wallBounded, xNodesMultiplier));  //
    }

    C.MatParamsMode = MaterialParamsMode.Sutherland;  //
    // Problem Definition
    //===================
    double L = 0.02; // separation between the two inlets, meters
    double TemperatureInFuel = 300;  //
    double TemperatureInOxidizer = 300; //
    double AtmPressure = 101325; // Pa
    double[] FuelInletConcentrations = new double[] { 0.2, 0.0, 0.0, 0.0, 0.8 };

    double[] OxidizerInletConcentrations = new double[] { 0.0, 0.23, 0.0, 0.0, 0.77 }; //
    double[] MWs = new double[] { CC.MW_CH4, CC.MW_O2, CC.MW_CO2, CC.MW_H2O, CC.MW_N2 };

    double mwFuel = CC.getAvgMW(MWs, FuelInletConcentrations);
    double mwAir = CC.getAvgMW(MWs, OxidizerInletConcentrations);
    double densityAirIn = AtmPressure * mwAir / (CC.R_gas * TemperatureInOxidizer * 1000); // kg / m3
    double densityFuelIn = AtmPressure * mwFuel / (CC.R_gas * TemperatureInFuel * 1000); // kg / m3.
    double uInFuel = massFuelIn / densityFuelIn; //
    double uInAir = massAirIn / densityAirIn; //

    Console.WriteLine("VelocityFuel" + uInFuel);
    Console.WriteLine("VelocityAir" + uInAir);

    // Reference values
    //===================
    // Basic units to be used: Kg, m, s, mol, pa,
    double TRef = TemperatureInOxidizer;// Reference temperature  is the inlet temperature, (K)
    double pRef = AtmPressure; // Pa
    double uRef = Math.Max(uInFuel, uInAir); // m/s
    double LRef = L;
    C.GravityDirection = new double[] { 0.0, 0.0, 0.0 }; //No gravity.

    // Solver configuration
    // =======================
    C.smoothingFactor = smoothing;
    // C.NonLinearSolver.ConvergenceCriterion = 1e-8;
    // C.LinearSolver.ConvergenceCriterion = 1e-10;

    C.NonLinearSolver.verbose = true;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.MaxSolverIterations = 10;
    C.LinearSolver = LinearSolverCode.direct_pardiso.GetConfig();
    C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
    C.saveperiod = 1;
    C.PenaltyViscMomentum = 1.0; 
    C.PenaltyHeatConduction = 1.0;
    C.YFuelInlet = FuelInletConcentrations[0];
    C.YOxInlet = OxidizerInletConcentrations[1];
    C.FuelInletConcentrations = FuelInletConcentrations;
    C.OxidizerInletConcentrations = OxidizerInletConcentrations;
    C.TFuelInlet = 1.0;
    C.TOxInlet = 1.0;
    C.PhysicalParameters.IncludeConvection = true;
    // Chemical related parameters
    double s = (CC.nu_O2 * CC.MW_O2) / (CC.nu_CH4 * CC.MW_CH4);
    C.phi = s * C.YFuelInlet / C.YOxInlet;
    C.zSt = 1.0 / (1.0 + C.phi);
    var MLC = new MaterialLawCombustion(300, new double[] { }, C.MatParamsMode, C.rhoOne, true, 1.0, 1, 1, C.YOxInlet, C.YFuelInlet, C.zSt, CC, 0.75);
    var ThermoProperties = new ThermodynamicalProperties();

    //==========================
    //Derived reference values
    //==========================
    C.uRef = uRef; // Reference velocity
    C.LRef = LRef; // reference length
    C.pRef = AtmPressure; // reference pressure
    C.TRef = TemperatureInFuel;// reference temperature
    C.MWRef = MLC.getAvgMW(MWs, C.OxidizerInletConcentrations); // Air mean molecular weight
    C.rhoRef = C.pRef * C.MWRef / (8.314 * C.TRef * 1000); // Kg/m3. ok ;
    C.cpRef = 1.3;//ThermoProperties.Calculate_Cp_Mixture(new double[] { 0.23, 0.77 }, new string[] { "O2", "N2" }, 300); // 1.219185317353029;// Representative value, KJ/Kg K ========> 1.31 for the one-step kinetic model
    C.muRef = MLC.getViscosityDim(300);
    C.MolarMasses = new double[] { C.CC.MW_CH4, C.CC.MW_O2, C.CC.MW_CO2, C.CC.MW_H2O, C.CC.MW_N2 };
    C.MolarMasses.ScaleV(1.0 / C.MWRef); //NonDimensionalized Molar masses
    C.T_ref_Sutherland = 300;
    double heatRelease_Ref = (C.TRef * C.cpRef);
    C.HeatRelease = C.CC.HeatReleaseMass / heatRelease_Ref;
    C.B = CC.PreExponentialFactor;

    C.StoichiometricCoefficients = new double[] { -1, -2, 1, 2, 0 };

    C.Damk = C.rhoRef * C.LRef * C.B / (C.uRef * C.MWRef);
    C.Reynolds = C.rhoRef * C.uRef * C.LRef / C.muRef;
    C.Prandtl = 0.75; 
    C.Schmidt = C.Prandtl; // Because Lewis number  is assumed as 1.0  (Le = Pr/Sc)
    // C.Lewis = new double[] { 0.97, 1.11, 1.39, 0.83, 1.0 };
    C.Lewis = new double[] { 1.0, 1.0, 1.0, 1.0, 1.0 };

    double g = 9.8; // m/s2
    C.Froude = Math.Sqrt(uRef * uRef / (C.LRef * g)); // Not used
    C.T_ref_Sutherland = 300; //////// Check this
    C.ReactionRateConstants = new double[] { C.Damk, CC.Ta / TRef, 1.0, 1.0 }; 
    //==========================
    // Initial conditions
    //==========================
    double dummy = 0;
    double Radius = 0.5;
    C.AddInitialValue(VariableNames.VelocityX, BoundaryValueFactory.Get_ConstantValue(0.0, Radius, uInFuel / C.uRef, uInAir / C.uRef,dummy));
    C.AddInitialValue(VariableNames.VelocityY, BoundaryValueFactory.Get_ConstantValue(0.0, Radius, uInFuel / C.uRef, uInAir / C.uRef,dummy));
    C.AddInitialValue(VariableNames.Pressure, BoundaryValueFactory.Get_ConstantValue(0.0, Radius, uInFuel / C.uRef, uInAir / C.uRef,dummy));

    //==========================
    // Boundary conditions
    //==========================
    double sigma = 10;
    if(parabolicVelocityProfile) {
        C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ParabolaVelocityFuel(dummy, Radius, uInFuel / C.uRef, dummy, dummy));
        C.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ParabolaVelocityAir(dummy, Radius, dummy, uInAir / C.uRef, dummy));
    } else { // Plug Flow
        // C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue(uInFuel / C.uRef, dummy, dummy, dummy, dummy));
        // C.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue((-1) * uInAir / C.uRef, dummy, dummy, dummy, dummy));
        C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_RegularizedPlugFlowFuel(dummy, Radius, uInFuel / C.uRef, dummy, sigma));
        C.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_RegularizedPlugFlowAir( dummy, Radius, dummy,  uInAir / C.uRef, sigma));
    }

    C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy,dummy));
    C.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy,dummy));
    if(wallBounded){
        C.AddBoundaryValue("Wall", VariableNames.VelocityX, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy,dummy));
        C.AddBoundaryValue("Wall", VariableNames.VelocityY, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy,dummy));
    }
    return C;
}

## Starting the MixtureFraction simulation
Configuration  for the simulation using the mixture fraction approach, where an infinite reaction rate is assumed. Used to find adequate starting solution for the full problem.

In [ ]:
static XNSEC_Control GiveMeTheMixtureFractionCtrlFile(int dg, int nCells, double massFuelIn, double massAirIn, bool parabolicVelocityProfile, int multiplier, bool useFullGeometry, bool wallBounded, double xNodesMultiplier, double smoothing){
    var C_MixtureFraction = GiveMeTheCtrlFile(dg, nCells, true,massFuelIn, massAirIn, parabolicVelocityProfile, useFullGeometry,  wallBounded , xNodesMultiplier, smoothing);
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.ProjectName = "CounterDifFlame";
    string name = C_MixtureFraction.ProjectName + "P" + dg + "K" + nCells + "Multiplier" +multiplier + "xNodesMultiplier"+xNodesMultiplier+"smoothing"+smoothing+"a";
    C_MixtureFraction.SessionName = "FS_" + name;

    C_MixtureFraction.UseSelfMadeTemporalOperator = false;
    C_MixtureFraction.ChemicalReactionActive = false;
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.NonLinearSolver.MaxSolverIterations = 50; 
    C_MixtureFraction.dummycounter = multiplier;
    
    // Boundary and initial conditions
    double dummy = -11111111;
    C_MixtureFraction.AddInitialValue(VariableNames.MixtureFraction,BoundaryValueFactory.Get_ConstantValue(1.0,dummy,dummy , dummy, dummy));
    C_MixtureFraction.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantValue(1.0,dummy,dummy , dummy, dummy));
    C_MixtureFraction.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantValue(0.0,dummy,dummy , dummy, dummy));

    C_MixtureFraction.Tags.Add(multiplier.ToString()); // Tag used for restart of the full problem
    double radius_inlet = 0.5; // radius is 1/2 from length separation
    var troubledPoints = new List<double[]>();
    if(useFullGeometry){
        troubledPoints.Add(new double[]{2, +radius_inlet });
        troubledPoints.Add(new double[]{2, -radius_inlet });
        troubledPoints.Add(new double[]{1, +radius_inlet });
        troubledPoints.Add(new double[]{1, -radius_inlet });
    } else {
        troubledPoints.Add(new double[]{0, +radius_inlet });
        troubledPoints.Add(new double[]{0, -radius_inlet });
        troubledPoints.Add(new double[]{1, +radius_inlet });
        troubledPoints.Add(new double[]{1, -radius_inlet });
    }
    

    bool useHomotopy = false;

    if(useHomotopy) { 
        C_MixtureFraction.HomotopyApproach = XNSEC_Control.HomotopyType.Automatic;
        // C_MixtureFraction.HomotopyVariable = XNSEC_Control.HomotopyVariableEnum.VelocityInletMultiplier;
        // C_MixtureFraction.homotopieAimedValue = multiplier;
        C_MixtureFraction.HomotopyVariable = XNSEC_Control.HomotopyVariableEnum.Reynolds;
        C_MixtureFraction.homotopieAimedValue = C_MixtureFraction.Reynolds;
    }


    C_MixtureFraction.AdaptiveMeshRefinement = true;
    C_MixtureFraction.AMR_startUpSweeps =2;
    int NoOfPseudoTimesteps = 8;
    C_MixtureFraction.TimesteppingMode = BoSSS.Solution.Control.AppControl._TimesteppingMode.Steady; 
    C_MixtureFraction.NoOfTimesteps = NoOfPseudoTimesteps ;

    // C_MixtureFraction.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onProblematicPoints(troubledPoints,C_MixtureFraction.AMR_startUpSweeps) );  
    C_MixtureFraction.activeAMRlevelIndicators.Add(new BoSSS.Application.XNSEC.AMR_RefineAroundProblematicPoints(troubledPoints, 1, 0.01));
    C_MixtureFraction.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onFlameSheet(C_MixtureFraction.zSt,8) );
    return C_MixtureFraction;
}

## Send and run jobs

In [ ]:
foreach(double xDensityMult in  xCellsMultiplier)
foreach(int nCells in nCellsArray){

foreach(int i in multiplierS){
  foreach(double smooth in Smoothings) {
  double massFuelIn = InitialMassFuelIn*i;
  double massAirIn = InitialMassAirIn*i;

  Type solver_MF = typeof(BoSSS.Application.XNSEC.XNSEC_MixtureFraction);
  var C_MixtureFraction = GiveMeTheMixtureFractionCtrlFile(dgMF, nCells, massFuelIn, massAirIn,  parabolicVelocityProfile, i, useFullGeometry, wallBounded, xDensityMult,smooth);
  string jobName       = C_MixtureFraction.SessionName;
  Console.WriteLine(jobName);
  var oneJob           = new Job(jobName, solver_MF);
  oneJob.NumberOfMPIProcs =  12;
  oneJob.SetControlObject(C_MixtureFraction);
    // oneJob.Activate(myBatch); 
    oneJob.Activate(); 
}
}
}

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

## Starting the finite-rate chemistry simulation

Now that the simulation for an "infinite" reaction rate is done, we use it for initializing the system with finite reaction rate.

In [ ]:
static XNSEC_Control GiveMeTheFullCtrlFile(int dg, int nCells, double massFuelIn, double massAirIn, ISessionInfo SessionToRestart, bool parabolicVelocityProfile, bool chemReactionActive, bool useFullGeometry, bool wallBounded, int mult, int counter, double xNodesMultiplier, double smoothing) {
    var C_OneStep = GiveMeTheCtrlFile(dg, nCells, false, massFuelIn, massAirIn, parabolicVelocityProfile, useFullGeometry, wallBounded, xNodesMultiplier, smoothing);
    C_OneStep.physicsMode = PhysicsMode.Combustion;
    C_OneStep.ProjectName = "CounterDifFlame";
    string name = C_OneStep.ProjectName + "P" + dg + "K" + nCells + "mult" + mult+"_c"+counter+"smooth"+smoothing ;
    C_OneStep.SessionName = "Full_" + name;
    C_OneStep.VariableOneStepParameters = false;

    // C_OneStep.Tags.Add("VelocityMultiplier" + mult);
    C_OneStep.Tags.Add( mult.ToString());
    C_OneStep.dummycounter = counter;
    C_OneStep.UseSelfMadeTemporalOperator = false;
    C_OneStep.myThermalWallType = SIPDiffusionTemperature.ThermalWallType.Adiabatic;
    C_OneStep.Timestepper_LevelSetHandling = BoSSS.Solution.XdgTimestepping.LevelSetHandling.None;
    C_OneStep.UseMixtureFractionsForCombustionInitialization = true;

    C_OneStep.LinearSolver = new BoSSS.Solution.AdvancedSolvers.OrthoMGSchwarzConfig() {
        NoOfMultigridLevels = 5,
        //verbose = true
    };

    C_OneStep.ChemicalReactionActive = chemReactionActive;
    C_OneStep.AdaptiveMeshRefinement = true;
    
    C_OneStep.HeatCapacityMode = MaterialLaw_MultipleSpecies.CpCalculationMode.constant;

    bool AMRinEachNewtonStep = false;

    if( AMRinEachNewtonStep) {
        C_OneStep.NoOfTimesteps = 4;
        C_OneStep.NonLinearSolver.MaxSolverIterations = 8; // Do only one newton iteration before refining
        C_OneStep.NonLinearSolver.MinSolverIterations = 8; // Do only one newton iteration before refining
    } else{
        C_OneStep.NoOfTimesteps = 3; // The steady solution will be calculated again and do AMR
        C_OneStep.NonLinearSolver.MaxSolverIterations = 3;
    }

    C_OneStep.AMR_startUpSweeps = 0;
    if(C_OneStep.ChemicalReactionActive){
        C_OneStep.activeAMRlevelIndicators.Add(new AMR_onReactiveZones( 2, 0.2));
        C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnVariableLimits("Temperature", new double[] { -100, 4 },3)); // Refine all cells with T > 5 (and T < -100)
        // C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnFieldGradient(2, 0.9, VariableNames.Temperature));
        // C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnPerssonSensor(VariableNames.Temperature, 2));
    }
    // C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnPerssonSensor(VariableNames.Temperature, 3));
    // C_OneStep.NonLinearSolver.MaxSolverIterations = 10;

    // limiting of variable values
    Dictionary<string, Tuple<double, double>> Bounds = new Dictionary<string, Tuple<double, double>>();
    double eps = 1e-2;
    Bounds.Add(VariableNames.Temperature, new Tuple<double, double>(1.0 - eps, 10)); // Min temp should be the inlet temperature.
    Bounds.Add(VariableNames.MassFraction0, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4)); // Between 0 and 1 per definition
    Bounds.Add(VariableNames.MassFraction1, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4));
    Bounds.Add(VariableNames.MassFraction2, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4));
    Bounds.Add(VariableNames.MassFraction3, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4));
    C_OneStep.VariableBounds = Bounds;
    // Boundary conditions

    double dummy = 0;
    if(SessionToRestart != null) {
        C_OneStep.SetRestart(SessionToRestart);
    } else {
        C_OneStep.AddInitialValue(VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(0.23, dummy, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
    }
    if(wallBounded){
        C_OneStep.AddBoundaryValue("Wall", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy, dummy));
    }
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[0], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[1], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[2], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[3], dummy, dummy, dummy, dummy));

    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[0], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[1], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[2], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[3], dummy, dummy, dummy, dummy));
    return C_OneStep;
}

In [ ]:
Type solver = typeof(BoSSS.Application.XNSEC.XNSEC);

Calculate the full solution for the initial value

In [ ]:
// int counter = 0;
// foreach (double xDensityNodes in xCellsMultiplier){
//     foreach (int nCells in nCellsArray) {
//         foreach (int dg in DGDegrees) {
//             foreach(double smooth in Smoothings){
//             foreach (int i in multiplierS) {
//                 var sess = (myDb.Sessions.Where(s => s.Name == "FS_CounterDifFlameP" + dgMF + "K" + nCells + "Multiplier" + i + "xNodesMultiplier" + xDensityNodes+"smoothing"+smooth)).FirstOrDefault();
//                 var C = GiveMeTheFullCtrlFile(dg, nCells, InitialMassFuelIn * i, InitialMassAirIn * i, sess, parabolicVelocityProfile, chemicalReactionActive, useFullGeometry, wallBounded, i, counter, xDensityNodes,smooth);
//                 string jobName = C.SessionName;
//                 Console.WriteLine(jobName);
//                 var oneJob = new Job(jobName, solver);
//                 oneJob.NumberOfMPIProcs = 12;
//                 oneJob.SetControlObject(C);
//                   // oneJob.Activate(myBatch); 
//                 oneJob.Activate();
//                 counter++;
//             }
//         }
//     }
// }
// }

In [ ]:
// wait for all jobs to finish (up to 2 days, check every 10 minutes)
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600*24*2), PollingIntervallSeconds:(60*10));

In [ ]:
// detect failed Jobs in the job management
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
    .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
                  || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
suspects

In [ ]:
NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

In [ ]:
// // Delete all MF calculations
// Console.WriteLine("Deleting mass fraction calculations");
// myDb.Sessions.Where(sess => sess.Name.Contains("FS")).ForEach(sess=> sess.Delete(true))

In [ ]:
var tsFULL = myDb.Sessions[5].Timesteps.Last();
var tsMF = myDb.Sessions[3].Timesteps.Last();

DGField TemperatureFULL       = tsFULL.Fields.Where(f => f.Identification == "Temperature").SingleOrDefault();
DGField TemperatureMF       = tsMF .Fields.Where(f => f.Identification == "Temperature").SingleOrDefault();

DGField Y0Full       = tsFULL.Fields.Where(f => f.Identification == "MassFraction0").SingleOrDefault();
DGField Y0MF       = tsMF .Fields.Where(f => f.Identification == "MassFraction0").SingleOrDefault();

DGField Y1Full       = tsFULL.Fields.Where(f => f.Identification == "MassFraction1").SingleOrDefault();
DGField Y1MF       = tsMF .Fields.Where(f => f.Identification == "MassFraction1").SingleOrDefault();


In [ ]:
double[] X =  GenericBlas.Linspace(0.0001, 1.0-0.0001,2000);
double[] TFull = new double[X.Length];
double[] TMF = new double[X.Length];
double[] TDIFF = new double[X.Length];

double[] Y0FullARR = new double[X.Length];
double[] Y0MFARR = new double[X.Length];

double[] Y1FullARR = new double[X.Length];
double[] Y1MFARR = new double[X.Length];


  

  

In [ ]:
for(int i  = 0; i < X.Length; i++){
    TFull[i] = TemperatureFULL.ProbeAt(new double[] {X[i],0.0001});
    TMF[i] = TemperatureMF.ProbeAt(new double[] {X[i],0.0001});
    Y0FullARR[i] = Y0Full.ProbeAt(new double[] {X[i],0.0001});
    Y0MFARR[i] = Y0MF.ProbeAt(new double[] {X[i],0.0001});
    Y1FullARR[i] = Y1Full.ProbeAt(new double[] {X[i],0.0001});
    Y1MFARR[i] = Y1MF.ProbeAt(new double[] {X[i],0.0001});
}

In [ ]:
var plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines;
plot.AddDataGroup("full",X,TFull, fmt);
plot.AddDataGroup("mf", X,TMF, fmt);

// plot.AddDataGroup("diff", X,TDIFF, fmt); 
plot . PlotNow ()


Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 0.6 
 
 
 
 
 0.7 
 
 
 
 
 0.8 
 
 
 
 
 0.9 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 full 
 
 
 full 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M632.9,57.1 L686.3,57.1 M37.4,432.0 L37.7,432.0 L38.1,432.0 L38.4,432.0 L38.8,432.0 L39.2,432.0
 L39.5,432.0 L39.9,432.0 L40.2,432.0 L40.6,432.0 L40.9,432.0 L41.3,432.0 L41.6,432.0 L42.0,432.0
 L42.4,432.0 L42.7,432.0 L43.1,432.0 L43.4,432.0 L43.8,432.0 L44.1,432.0 L44.5,432.0 L44.9,432.0
 L45.2,432.0 L45.6,432.0 L45.9,432.0 L46.3,432.0 L46.6,432.0 L47.0,432.0 L47.4,432.0 L47.7,432.0
 L48.1,432.0 L48.4,432.0 L48.8,432.0 L49.1,432.0 L49.5,432.0 L49.8,432.0 L50.2,432.0 L50.6,432.0
 L50.9,432.0 L51.3,432.0 L51.6,432.0 L52.0,432.0 L52.3,432.0 L52.7,432.0 L53.1,432.0 L53.4,432.0
 L53.8,432.0 L54.1,432.0 L54.5,432.0 L54.8,432.0 L55.2,432.0 L55.6,432.0 L55.9,432.0 L56.3,432.0
 L56.6,432.0 L57.0,432.0 L57.3,432.0 L57.7,432.0 L58.0,432.0 L58.4,432.0 L58.8,432.0 L59.1,432.0
 L59.5,432.0 L59.8,432.0 L60.2,432.0 L60.5,432.0 L60.9,432.0 L61.3,432.0 L61.6,432.0 L62.0,432.0
 L62.3,432.0 L62.7,432.0 L63.0,432.0 L63.4,432.0 L63.8,432.0 L64.1,432.0 L64.5,432.0 L64.8,432.0
 L65.2,432.0 L65.5,432.0 L65.9,432.0 L66.2,432.0 L66.6,432.0 L67.0,432.0 L67.3,432.0 L67.7,432.0
 L68.0,432.0 L68.4,432.0 L68.7,432.0 L69.1,432.0 L69.5,432.0 L69.8,432.0 L70.2,432.0 L70.5,432.0
 L70.9,432.0 L71.2,432.0 L71.6,432.0 L72.0,432.0 L72.3,432.0 L72.7,432.0 L73.0,432.0 L73.4,432.0
 L73.7,432.0 L74.1,432.0 L74.4,432.0 L74.8,432.0 L75.2,432.0 L75.5,432.0 L75.9,432.0 L76.2,432.0
 L76.6,432.0 L76.9,432.0 L77.3,432.0 L77.7,432.0 L78.0,432.0 L78.4,432.0 L78.7,432.0 L79.1,432.0
 L79.4,432.0 L79.8,432.0 L80.2,432.0 L80.5,432.0 L80.9,432.0 L81.2,432.0 L81.6,432.0 L81.9,432.0
 L82.3,432.0 L82.6,432.0 L83.0,432.0 L83.4,432.0 L83.7,432.0 L84.1,432.0 L84.4,432.0 L84.8,432.0
 L85.1,432.0 L85.5,432.0 L85.9,432.0 L86.2,432.0 L86.6,432.0 L86.9,432.0 L87.3,432.0 L87.6,432.0
 L88.0,432.0 L88.4,432.0 L88.7,432.0 L89.1,432.0 L89.4,432.0 L89.8,432.0 L90.1,432.0 L90.5,432.0
 L90.8,432.0 L91.2,432.0 L91.6,432.0 L91.9,432.0 L92.3,432.0 L92.6,432.0 L93.0,432.0 L93.3,432.0
 L93.7,432.0 L94.1,432.0 L94.4,432.0 L94.8,432.0 L95.1,432.0 L95.5,432.0 L95.8,432.0 L96.2,432.0
 L96.6,432.0 L96.9,432.0 L97.3,432.0 L97.6,432.0 L98.0,432.0 L98.3,432.0 L98.7,432.0 L99.0,432.0
 L99.4,432.0 L99.8,432.0 L100.1,432.0 L100.5,432.0 L100.8,432.0 L101.2,432.0 L101.5,432.0 L101.9,432.0
 L102.3,432.0 L102.6,432.0 L103.0,432.0 L103.3,432.0 L103.7,432.0 L104.0,432.0 L104.4,432.0 L104.8,432.0
 L105.1,432.0 L105.5,432.0 L105.8,432.0 L106.2,432.0 L106.5,432.0 L106.9,432.0 L107.2,432.0 L107.6,432.0
 L108.0,432.0 L108.3,432.0 L108.7,432.0 L109.0,432.0 L109.4,432.0 L109.7,432.0 L110.1,432.0 L110.5,432.0
 L110.8,432.0 L111.2,432.0 L111.5,432.0 L111.9,432.0 L112.2,432.0 L112.6,432.0 L113.0,432.0 L113.3,432.0
 L113.7,432.0 L114.0,432.0 L114.4,432.0 L114.7,432.0 L115.1,432.0 L115.4,432.0 L115.8,432.0 L116.2,432.0
 L116.5,432.0 L116.9,432.0 L117.2,432.0 L117.6,432.0 L117.9,432.0 L118.3,432.0 L118.7,432.0 L119.0,432.0
 L119.4,432.0 L119.7,432.0 L120.1,432.0 L120.4,432.0 L120.8,432.0 L121.2,432.0 L121.5,432.0 L121.9,432.0
 L122.2,431.9 L122.6,431.9 L122.9,431.9 L123.3,431.9 L123.6,431.9 L124.0,431.9 L124.4,431.9 L124.7,431.9
 L125.1,431.9 L125.4,431.9 L125.8,431.9 L126.1,431.9 L126.5,431.9 L126.9,431.9 L127.2,431.9 L127.6,431.9
 L127.9,431.9 L128.3

In [ ]:
var plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines;

plot.AddDataGroup("full",X,Y0FullARR, fmt);
plot.AddDataGroup("mf", X,Y0MFARR, fmt);
plot.AddDataGroup("full",X,Y1FullARR, fmt);
plot.AddDataGroup("mf", X,Y1MFARR, fmt);
// plot.AddDataGroup("diff", X,TDIFF, fmt); 
plot . PlotNow ()


Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.05 
 
 
 
 
 0 
 
 
 
 
 0.05 
 
 
 
 
 0.1 
 
 
 
 
 0.15 
 
 
 
 
 0.2 
 
 
 
 
 0.25 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 0.6 
 
 
 
 
 0.7 
 
 
 
 
 0.8 
 
 
 
 
 0.9 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 full 
 
 
 full 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M632.9,57.1 L686.3,57.1 M62.3,124.1 L62.6,124.1 L63.0,124.1 L63.3,124.1 L63.6,124.1 L64.0,124.1
 L64.3,124.1 L64.7,124.1 L65.0,124.1 L65.4,124.1 L65.7,124.1 L66.1,124.1 L66.4,124.1 L66.7,124.1
 L67.1,124.1 L67.4,124.1 L67.8,124.1 L68.1,124.1 L68.5,124.1 L68.8,124.1 L69.1,124.1 L69.5,124.1
 L69.8,124.1 L70.2,124.1 L70.5,124.1 L70.9,124.1 L71.2,124.1 L71.6,124.1 L71.9,124.1 L72.2,124.1
 L72.6,124.1 L72.9,124.1 L73.3,124.1 L73.6,124.1 L74.0,124.1 L74.3,124.1 L74.7,124.1 L75.0,124.1
 L75.3,124.1 L75.7,124.1 L76.0,124.1 L76.4,124.1 L76.7,124.1 L77.1,124.1 L77.4,124.1 L77.8,124.1
 L78.1,124.1 L78.4,124.1 L78.8,124.1 L79.1,124.1 L79.5,124.1 L79.8,124.1 L80.2,124.1 L80.5,124.1
 L80.8,124.1 L81.2,124.1 L81.5,124.1 L81.9,124.1 L82.2,124.1 L82.6,124.1 L82.9,124.1 L83.3,124.1
 L83.6,124.1 L83.9,124.1 L84.3,124.1 L84.6,124.1 L85.0,124.1 L85.3,124.1 L85.7,124.1 L86.0,124.1
 L86.4,124.1 L86.7,124.1 L87.0,124.1 L87.4,124.1 L87.7,124.1 L88.1,124.1 L88.4,124.1 L88.8,124.1
 L89.1,124.1 L89.4,124.1 L89.8,124.1 L90.1,124.1 L90.5,124.1 L90.8,124.1 L91.2,124.1 L91.5,124.1
 L91.9,124.1 L92.2,124.1 L92.5,124.1 L92.9,124.1 L93.2,124.1 L93.6,124.1 L93.9,124.1 L94.3,124.1
 L94.6,124.1 L95.0,124.1 L95.3,124.1 L95.6,124.1 L96.0,124.1 L96.3,124.1 L96.7,124.1 L97.0,124.1
 L97.4,124.1 L97.7,124.1 L98.1,124.1 L98.4,124.1 L98.7,124.1 L99.1,124.1 L99.4,124.1 L99.8,124.1
 L100.1,124.1 L100.5,124.1 L100.8,124.1 L101.1,124.1 L101.5,124.1 L101.8,124.1 L102.2,124.1 L102.5,124.1
 L102.9,124.1 L103.2,124.1 L103.6,124.1 L103.9,124.1 L104.2,124.1 L104.6,124.1 L104.9,124.1 L105.3,124.1
 L105.6,124.1 L106.0,124.1 L106.3,124.1 L106.7,124.1 L107.0,124.1 L107.3,124.1 L107.7,124.1 L108.0,124.1
 L108.4,124.1 L108.7,124.1 L109.1,124.1 L109.4,124.1 L109.7,124.1 L110.1,124.1 L110.4,124.1 L110.8,124.1
 L111.1,124.1 L111.5,124.1 L111.8,124.1 L112.2,124.1 L112.5,124.1 L112.8,124.1 L113.2,124.1 L113.5,124.1
 L113.9,124.1 L114.2,124.1 L114.6,124.1 L114.9,124.1 L115.3,124.1 L115.6,124.1 L115.9,124.1 L116.3,124.1
 L116.6,124.1 L117.0,124.1 L117.3,124.1 L117.7,124.1 L118.0,124.1 L118.3,124.1 L118.7,124.1 L119.0,124.1
 L119.4,124.1 L119.7,124.1 L120.1,124.1 L120.4,124.1 L120.8,124.1 L121.1,124.1 L121.4,124.1 L121.8,124.1
 L122.1,124.1 L122.5,124.1 L122.8,124.1 L123.2,124.1 L123.5,124.1 L123.9,124.1 L124.2,124.1 L124.5,124.1
 L124.9,124.1 L125.2,124.1 L125.6,124.1 L125.9,124.1 L126.3,124.1 L126.6,124.1 L127.0,124.1 L127.3,124.1
 L127.6,124.1 L128.0,124.1 L128.3,124.1 L128.7,124.1 L129.0,124.1 L129.4,124.1 L129.7,124.1 L130.0,124.1
 L130.4,124.1 L130.7,124.1 L131.1,124.1 L131.4,124.1 L131.8,124.1 L132.1,124.1 L132.5,124.1 L132.8,124.1
 L133.1,124.1 L133.5,124.1 L133.8,124.1 L134.2,124.1 L134.5,124.0 L134.9,124.0 L135.2,124.0 L135.6,124.0
 L135.9,124.0 L136.2,124.0 L136.6,124.0 L136.9,124.0 L137.3,124.0 L137.6,124.0 L138.0,124.0 L138.3,124.0
 L138.6,124.0 L139.0,124.0 L139.3,124.0 L139.7,124.0 L140.0,124.0 L140.4,124.0 L140.7,124.0 L141.1,124.0
 L141.4,124.0 L141.7,124.0 L142.1,124.0 L142.4,124.0 L142.8,124.0 L143.1,124.1 L143.5,124.1 L143.8,124.1
 L144.2,124.1 L144.5,124.1 L144.8,124.1 L145.2,124.1 L145.5,124.1 L145.9,124.1 L146.2,124.1 L146.6,124.1
 L146.9,124.1 L147.2,124.1 L147.6,124.1 L147.9,124.1 L148.3,124.1 L148.6,124.1 L149.0,1

In [ ]:
myDb.Sessions

#0: CounterFlowFlame_MF_FullComparison_bugfixed155	FS_CounterDifFlameP3K10Multiplier5xNodesMultiplier1smoothing50a	04/05/2022 14:57:36	2dbda281...
#1: CounterFlowFlame_MF_FullComparison_bugfixed155	FS_CounterDifFlameP3K10Multiplier5xNodesMultiplier1smoothing100a	04/05/2022 14:48:17	a6795cae...
#2: CounterFlowFlame_MF_FullComparison_bugfixed155	FS_CounterDifFlameP3K10Multiplier5xNodesMultiplier1smoothing200a	04/05/2022 13:51:02	ee2d3ce0...
#3: CounterFlowFlame_MF_FullComparison_bugfixed155	FS_CounterDifFlameP3K10Multiplier5xNodesMultiplier1smoothing0a	04/05/2022 13:50:19	f470fbaf...


In [ ]:
List<double[]> sols = new List<double[]>();
for(int i = 0; i < Smoothings.Length; i++){
    DGField Temperature    =  myDb.Sessions[i].Timesteps.Last().Fields.Where(f => f.Identification == "Temperature").SingleOrDefault();
    double[] TMF = new double[X.Length];
    Console.WriteLine("A");
    for(int j  = 0; j < X.Length; j++){
        TMF[j] = Temperature.ProbeAt(new double[] {X[j],0.001});
    }
     sols.Add(TMF);    
}

A
A
A
A


In [ ]:
LineColors[] allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
var plot = new Plot2Ddata();
// for(int i = 0; i < Smoothings.Length; i++){
//         var fmt = new PlotFormat();
//     fmt.Style = Styles.Lines;
//     fmt.LineColor = allColors[i+1];
//     plot.AddDataGroup("smoothing "+1,X,sols[i], fmt);
// }
var fmt = new PlotFormat();
fmt.LineColor = BoSSS.Solution.Gnuplot.LineColors.Green;
plot.AddDataGroup("smoothing "+50,X,sols[1], fmt);

fmt = new PlotFormat();
fmt.LineColor = BoSSS.Solution.Gnuplot.LineColors.Black;
plot.AddDataGroup("smoothing "+100,X,sols[2], fmt);

fmt = new PlotFormat();
fmt.LineColor = BoSSS.Solution.Gnuplot.LineColors.Blue;
plot.AddDataGroup("smoothing "+200,X,sols[0], fmt);

// fmt = new PlotFormat();
// fmt.LineColor = BoSSS.Solution.Gnuplot.LineColors.Red;
// plot.AddDataGroup("full",X,TFull, fmt);

// fmt = new PlotFormat();
// fmt.LineColor = BoSSS.Solution.Gnuplot.LineColors.Magenta;
// plot.AddDataGroup("mf", X,TMF, fmt);

plot.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 0.6 
 
 
 
 
 0.7 
 
 
 
 
 0.8 
 
 
 
 
 0.9 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 smoothing 50 
 
 
 smoothing 50 
 
 
 
	<path stroke='rgb( 0, 255, 0)' d='M533.0,57.1 L586.4,57.1 M37.4,488.6 L37.7,488.6 L38.1,488.6 L38.4,488.6 L38.8,488.6 L39.2,488.6
 L39.5,488.6 L39.9,488.6 L40.2,488.6 L40.6,488.6 L40.9,488.6 L41.3,488.6 L41.6,488.6 L42.0,488.6
 L42.4,488.6 L42.7,488.6 L43.1,488.6 L43.4,488.6 L43.8,488.6 L44.1,488.6 L44.5,488.6 L44.9,488.6
 L45.2,488.6 L45.6,488.6 L45.9,488.6 L46.3,488.6 L46.6,488.6 L47.0,488.6 L47.4,488.6 L47.7,488.6
 L48.1,488.6 L48.4,488.6 L48.8,488.6 L49.1,488.6 L49.5,488.6 L49.8,488.6 L50.2,488.6 L50.6,488.6
 L50.9,488.6 L51.3,488.6 L51.6,488.6 L52.0,488.6 L52.3,488.6 L52.7,488.6 L53.1,488.6 L53.4,488.6
 L53.8,488.6 L54.1,488.6 L54.5,488.6 L54.8,488.6 L55.2,488.6 L55.6,488.6 L55.9,488.6 L56.3,488.6
 L56.6,488.6 L57.0,488.6 L57.3,488.6 L57.7,488.6 L58.0,488.6 L58.4,488.6 L58.8,488.6 L59.1,488.6
 L59.5,488.6 L59.8,488.6 L60.2,488.6 L60.5,488.6 L60.9,488.6 L61.3,488.6 L61.6,488.6 L62.0,488.6
 L62.3,488.6 L62.7,488.6 L63.0,488.6 L63.4,488.6 L63.8,488.6 L64.1,488.6 L64.5,488.6 L64.8,488.6
 L65.2,488.6 L65.5,488.6 L65.9,488.6 L66.2,488.6 L66.6,488.6 L67.0,488.6 L67.3,488.6 L67.7,488.6
 L68.0,488.6 L68.4,488.6 L68.7,488.6 L69.1,488.6 L69.5,488.6 L69.8,488.6 L70.2,488.6 L70.5,488.6
 L70.9,488.6 L71.2,488.6 L71.6,488.6 L72.0,488.6 L72.3,488.6 L72.7,488.6 L73.0,488.6 L73.4,488.6
 L73.7,488.6 L74.1,488.6 L74.4,488.6 L74.8,488.6 L75.2,488.6 L75.5,488.6 L75.9,488.6 L76.2,488.6
 L76.6,488.6 L76.9,488.6 L77.3,488.6 L77.7,488.6 L78.0,488.6 L78.4,488.6 L78.7,488.6 L79.1,488.6
 L79.4,488.6 L79.8,488.6 L80.2,488.6 L80.5,488.6 L80.9,488.6 L81.2,488.6 L81.6,488.6 L81.9,488.6
 L82.3,488.6 L82.6,488.6 L83.0,488.6 L83.4,488.6 L83.7,488.6 L84.1,488.6 L84.4,488.6 L84.8,488.6
 L85.1,488.6 L85.5,488.6 L85.9,488.6 L86.2,488.6 L86.6,488.6 L86.9,488.6 L87.3,488.6 L87.6,488.6
 L88.0,488.6 L88.4,488.6 L88.7,488.6 L89.1,488.6 L89.4,488.6 L89.8,488.6 L90.1,488.6 L90.5,488.6
 L90.8,488.6 L91.2,488.6 L91.6,488.6 L91.9,488.6 L92.3,488.6 L92.6,488.6 L93.0,488.6 L93.3,488.6
 L93.7,488.6 L94.1,488.6 L94.4,488.6 L94.8,488.6 L95.1,488.6 L95.5,488.6 L95.8,488.6 L96.2,488.6
 L96.6,488.6 L96.9,488.6 L97.3,488.6 L97.6,488.6 L98.0,488.6 L98.3,488.6 L98.7,488.6 L99.0,488.6
 L99.4,488.6 L99.8,488.6 L100.1,488.6 L100.5,488.6 L100.8,488.6 L101.2,488.6 L101.5,488.6 L101.9,488.6
 L102.3,488.6 L102.6,488.6 L103.0,488.6 L103.3,488.6 L103.7,488.6 L104.0,488.6 L104.4,488.6 L104.8,488.6
 L105.1,488.6 L105.5,488.6 L105.8,488.6 L106.2,488.6 L106.5,488.6 L106.9,488.6 L107.2,488.6 L107.6,488.6
 L108.0,488.6 L108.3,488.6 L108.7,489.0 L109.0,488.9 L109.4,488.9 L109.7,488.9 L110.1,488.8 L110.5,488.8
 L110.8,488.7 L111.2,488.7 L111.5,488.6 L111.9,488.6 L112.2,488.6 L112.6,488.5 L113.0,488.5 L113.3,488.5
 L113.7,488.4 L114.0,488.4 L114.4,488.4 L114.7,488.4 L115.1,488.3 L115.4,488.3 L115.8,488.3 L116.2,488.3
 L116.5,488.3 L116.9,488.2 L117.2,488.2 L117.6,488.2 L117.9,488.2 L118.3,488.2 L118.7,488.2 L119.0,488.2
 L119.4,488.2 L119.7,488.2 L120.1,488.2 L120.4,488.2 L120.8,488.1 L121.2,488.1 L121.5,488.1 L121.9,488.1
 L122.2,488.1 L122.6,488.1 L122.9,488.1 L123.3,488.1 L123.6,488.1 L124.0,488.1 L124.4,488.2 L124.7,488.2
 L125.1,488.2 L125.4,488.2 L125.8,488.2 L126.1,488.2 L126.5,488.2 L126.9,488.2 L127.2,488.2 L127.6,488.2
 L127.9,488.2 L128.3,488.2

In [ ]:
myDb.Sessions[0].Export().Do()

In [ ]:
string basepath = @"C:\tmp\MF_FULL_COMPARISON\";


In [ ]:

private static void ToTxtFile(string fileName, double[] x, double[] y) {
    if(x.Length != y.Length)
        throw new Exception();
        using (var file = new StreamWriter(fileName)) {
        for(int i = 0; i< x.Length; i++){
            file.Write(x[i] + "\t" + y[i]);
            file.WriteLine();
        }
    }    
}

In [ ]:

ToTxtFile(basepath+"TMF.txt",X,TMF);
ToTxtFile(basepath+"TFull.txt",X,TFull);


ToTxtFile(basepath+"Y0MF.txt",X,Y0MFARR);
ToTxtFile(basepath+"Y0Full.txt",X,Y0FullARR);


ToTxtFile(basepath+"Y1MF.txt",X,Y1MFARR);
ToTxtFile(basepath+"Y1Full.txt",X,Y1FullARR);

In [ ]:

ToTxtFile(basepath+"T_smooth50.txt",X,sols[0]);
ToTxtFile(basepath+"T_smooth100.txt",X,sols[1]);
ToTxtFile(basepath+"T_smooth200.txt",X,sols[2]);
ToTxtFile(basepath+"T_smooth0.txt",X,sols[3]);


In [ ]:
myDb.Sessions

#0: CounterFlowFlame_MF_FullComparison_bugfixed155	FS_CounterDifFlameP3K10Multiplier5xNodesMultiplier1smoothing50a	04/05/2022 14:57:36	2dbda281...
#1: CounterFlowFlame_MF_FullComparison_bugfixed155	FS_CounterDifFlameP3K10Multiplier5xNodesMultiplier1smoothing100a	04/05/2022 14:48:17	a6795cae...
#2: CounterFlowFlame_MF_FullComparison_bugfixed155	FS_CounterDifFlameP3K10Multiplier5xNodesMultiplier1smoothing200a	04/05/2022 13:51:02	ee2d3ce0...
#3: CounterFlowFlame_MF_FullComparison_bugfixed155	FS_CounterDifFlameP3K10Multiplier5xNodesMultiplier1smoothing0a	04/05/2022 13:50:19	f470fbaf...
